# 進階版踩地雷、找寶藏

原本的踩地雷只有一種地雷，而目的只是要找到沒有地雷的所有地方，我覺得有點像在自虐，因此設計了一個有多種地雷而且有更強目的性的小遊戲。

規則如下：  
玩家必須在一個15x15矩陣地圖上尋找寶藏，同時地圖中有三種爆炸範圍不同的地雷，玩家必須設法閃過。一開始地圖上各點皆初始化為零，當所選座標附近有地雷時值才會改變。如果玩家所選座標附近沒有包含任何爆炸範圍或寶藏則什麼事也不會發生。玩家需要輸入i、j座標，其中i、j分別代表矩陣中row與column的編號(其中0 <= i <= 14、0 <= j <= 14)。    
一號地雷：範圍是整個橫排，共有三個。  
二號地雷：範圍是整個直排，共有三個。  
三號地雷：範圍是以其為中心的九宮格，共有一個。  
當所輸入的座標附近(附近定義為「以輸入之座標為中心的九宮格內」)有重疊到任何一種地雷的爆炸範圍時，所選座標的值將改成該地雷的編號。例如所選座標附近重疊到一號地雷爆炸範圍則該點值改為1，如果同時涵蓋多個地雷爆炸範圍例如同時涵蓋一號及三號地雷則值改為13......以此類推。如果所輸入座標附近有寶藏，該點值則改為錢字號，並且該點即使附近有涵蓋到爆炸範圍也將只顯示錢字號，畢竟人類是短視而盲目的。  
玩家起初有五條命，當所輸入座標直接涵蓋到爆炸範圍(非附近)則引爆地雷並且扣一條命。被引爆的地雷在地圖上將顯示成X，即使地雷已經被引爆，踩到該地雷的爆炸範圍依然會扣生命值。(過程中地圖上較晚得到的資訊可能會蓋掉較早得到的資訊)  
在生命值被扣完前找到寶藏即為勝利。


In [4]:
import numpy as np
import pandas as pd
import random

MAP = np.zeros((15,15))
MAP_answer = np.zeros((15,15))
find = 0

x = np.array(random.sample(range(0,15),8))
y = np.array(random.sample(range(0,15),8))

for i in range(0,8):
   for j in range(i+1,8):
    if(x[i] == x[j] & y[i] == y[j]):
        if(y[j+1] != 14):
            y[j+1] += 1
        else:
            y[j+1] -= 1
            
MAP = np.zeros((15,15))
LIFE = 5

def printMap (MAP):
    for i in range(0,15):
        for j in range(0,15):
            if(j == 0):
                print("%6d"%i,end='')
                print("|",end='')
            if(MAP[i,j] != -1 and MAP[i,j] != 9):
                print("%6d"%MAP[i,j],end='')
            if(MAP[i,j] == -1 ):
                print("%6s"%"X",end='')
            if(MAP[i,j] == 9 ):
                print("%6s"%"$",end='')
            if(j == 14):
                print('\n')
    
    print("%13s"%"-",end='')
    for i in range(14):
        print("%6s"%"-",end='')
    print("\n")
    print("%6s"%"%",end='')
    print("|",end='')
    for k in range(0,15):
        print("%6d"%k,end='')

In [5]:
print('遊戲開始，此為起始地圖\n')
printMap(MAP)
while(LIFE > 0):
    i = input('輸入i座標')
    j = input('輸入j座標')
    type = np.array([0,0,0,0])
    
    
    if( (int(i) < 0) or (int(i) > 14) or (int(j) < 0) or int(j) > 14 ):
        print('所輸入座標超出地圖範圍，請重新輸入')
        continue
    
    #地雷一
    if( (x[0] - 1 <= int(i) <= x[0] + 1) or (x[1] - 1 <= int(i) <= x[1] + 1) or (x[2] - 1 <= int(i) <= x[2] + 1) ):
        type[1] = 1
        print(f'附近為地雷一的爆炸範圍')
    if( (int(i) == x[0]) or (int(i) == x[1]) or (int(i) == x[2])):
        LIFE -= 1
        for k in range(0,15):
            MAP[int(i),k] = -1
        print(f'踩到地雷一的爆炸範圍，生命值-1  剩餘生命：{LIFE}')
   
    #地雷二    
    if( (y[3] - 1 <= int(j) <= y[3] + 1) or (y[4] - 1 <= int(j) <= y[4] + 1) or (y[5] - 1 <= int(j) <= y[5] + 1) ):
        type[2] = 1
        print(f'附近為地雷二的爆炸範圍')
    if( (int(j) == y[3]) or (int(j) == y[4]) or (int(j) == y[5])):
        LIFE -= 1
        for k in range(0,15):
            MAP[k,int(j)] = -1
        print(f'踩到地雷二的爆炸範圍，生命值-1  剩餘生命：{LIFE}')
    
    #地雷三
    if( (x[6] - 2 <= int(i) <= x[6] + 2) and (y[6] - 2 <= int(j) <= y[6] + 2) ):
        type[3] = 1
        print(f'附近為地雷三的爆炸範圍')
    if( (x[6] - 1 <= int(i) <= x[6] + 1) and (y[6] - 1 <= int(j) <= y[6] + 1) ):
        LIFE -= 1
        for k in range(x[6] - 1, x[6] + 2):
            for l in range(y[6] - 1, y[6] + 2):
                if(0 <= k <= 14 and 0 <= l <= 14):
                    MAP[k,l] = -1
        print(f'踩到地雷三的爆炸範圍，生命值-1  剩餘生命：{LIFE}')
        
    #寶藏
    if( (x[7] - 1 <= int(i) <= x[7] + 1) and (y[7] - 1 <= int(j) <= y[7] + 1) ):
        type[0] = 1
    if( (int(i) == x[7]) and (int(j) == y[7]) ):
        print(f'恭喜找到寶藏！！！！！！！！！')
        find = 1
    if( (x[7] - 1 <= int(i) <= x[7] + 1) and (y[7] - 1 <= int(j) <= y[7] + 1) and (find != 1) ):
        print(f'!!!!!附近有寶藏!!!!!')
        
        
    #modify
    if(type[1] == 1 and type[2] == 0 and type[3] == 0):
        MAP[int(i),int(j)] = 1
    if(type[1] == 0 and type[2] == 1 and type[3] == 0):
        MAP[int(i),int(j)] = 2
    if(type[1] == 0 and type[2] == 0 and type[3] == 1):
        MAP[int(i),int(j)] = 3
      
    
    if(type[1] == 1 and type[2] == 1 and type[3] == 0):
        MAP[int(i),int(j)] = 12
    if(type[1] == 0 and type[2] == 1 and type[3] == 1):
        MAP[int(i),int(j)] = 23
    if(type[1] == 1 and type[2] == 0 and type[3] == 1):
        MAP[int(i),int(j)] = 13
    
    if(type[1] == 1 and type[2] == 1 and type[3] == 1):
        MAP[int(i),int(j)] = 123
    
    if(type[0] == 1):
        MAP[int(i),int(j)] = 9
        
    if(find == 1):
        break
        
    printMap(MAP)
        

print("\n")   
if(LIFE <= 0 and find == 0):
    print('生命歸零，挑戰失敗')
if(LIFE > 0 and find == 1):
    print('挑戰成功')
if(LIFE <= 0 and find == 1):
    print('找到寶藏，但生命已歸零，挑戰失敗')


    
    

遊戲開始，此為起始地圖

     0|     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0

     1|     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0

     2|     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0

     3|     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0

     4|     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0

     5|     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0

     6|     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0

     7|     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0

     8|     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0

     9|     0     0     0     0     0     0     0     0     0     0     0     0     0     0     

輸入i座標 3
輸入j座標 10


附近為地雷一的爆炸範圍
踩到地雷一的爆炸範圍，生命值-1  剩餘生命：4
附近為地雷二的爆炸範圍
踩到地雷二的爆炸範圍，生命值-1  剩餘生命：3
     0|     0     0     0     0     0     0     0     0     0     0     X     0     0     0     0

     1|     0     0     0     0     0     0     0     0     0     0     X     0     0     0     0

     2|     0     0     0     0     0     0     0     0     0     0     X     0     0     0     0

     3|     X     X     X     X     X     X     X     X     X     X    12     X     X     X     X

     4|     0     0     0     0     0     0     0     0     0     0     X     0     0     0     0

     5|     0     0     0     0     0     0     0     0     0     0     X     0     0     0     0

     6|     0     0     0     0     0     0     0     0     0     0     X     0     0     0     0

     7|     0     0     0     0     0     0     0     0     0     0     X     0     0     0     0

     8|     0     0     0     0     0     0     0     0     0     0     X     0     0     0     0

     9|     0     0     0     0    

輸入i座標 8
輸入j座標 8


     0|     0     0     0     0     0     0     0     0     0     0     X     0     0     0     0

     1|     0     0     0     0     0     0     0     0     0     0     X     0     0     0     0

     2|     0     0     0     0     0     0     0     0     0     0     X     0     0     0     0

     3|     X     X     X     X     X     X     X     X     X     X    12     X     X     X     X

     4|     0     0     0     0     0     0     0     0     0     0     X     0     0     0     0

     5|     0     0     0     0     0     0     0     0     0     0     X     0     0     0     0

     6|     0     0     0     0     0     0     0     0     0     0     X     0     0     0     0

     7|     0     0     0     0     0     0     0     0     0     0     X     0     0     0     0

     8|     0     0     0     0     0     0     0     0     0     0     X     0     0     0     0

     9|     0     0     0     0     0     0     0     0     0     0     X     0     0     0     0

    10|   

...

附近為地雷二的爆炸範圍
踩到地雷二的爆炸範圍，生命值-1  剩餘生命：1
!!!!!附近有寶藏!!!!!
     0|     0     0     0     0     0     0     0     0     0     0     X     0     0     X     0

     1|     0     0     0     0     0     0     0     0     0     0     X     0     0     X     0

     2|     0     0     0     0     0     0     0     0     0     0     X     0     0     X     0

     3|     X     X     X     X     X     1     X     X     X     X     X     X     X     X     X

     4|     0     0     0     0     0     0     0     0     0     0     X     0     0     X     0

     5|     0     0     0     0     0     0     0     0     0     0     X     0     0     X     0

     6|     0     0     0     0     0     0     0     0     0     0     X     0     0     X     0

     7|     0     0     0     0     0     0     0     0     0     0     X     0     0     X     0

     8|     0     0     0     0     0     0     0     0     0     0     X     0     0     X     0

     9|     0     0     0     0     0     0     0     0 

輸入i座標 12
輸入j座標 12


附近為地雷二的爆炸範圍
!!!!!附近有寶藏!!!!!
     0|     0     0     0     0     0     0     0     0     0     0     X     0     0     X     0

     1|     0     0     0     0     0     0     0     0     0     0     X     0     0     X     0

     2|     0     0     0     0     0     0     0     0     0     0     X     0     0     X     0

     3|     X     X     X     X     X     1     X     X     X     X     X     X     X     X     X

     4|     0     0     0     0     0     0     0     0     0     0     X     0     0     X     0

     5|     0     0     0     0     0     0     0     0     0     0     X     0     0     X     0

     6|     0     0     0     0     0     0     0     0     0     0     X     0     0     X     0

     7|     0     0     0     0     0     0     0     0     0     0     X     0     0     X     0

     8|     0     0     0     0     0     0     0     0     0     0     X     0     0     X     0

     9|     0     0     0     0     0     0     0     0     0     0     X     0  

輸入i座標 12
輸入j座標 11


附近為地雷二的爆炸範圍
踩到地雷二的爆炸範圍，生命值-1  剩餘生命：0
!!!!!附近有寶藏!!!!!
     0|     0     0     0     0     0     0     0     0     0     0     X     X     0     X     0

     1|     0     0     0     0     0     0     0     0     0     0     X     X     0     X     0

     2|     0     0     0     0     0     0     0     0     0     0     X     X     0     X     0

     3|     X     X     X     X     X     1     X     X     X     X     X     X     X     X     X

     4|     0     0     0     0     0     0     0     0     0     0     X     X     0     X     0

     5|     0     0     0     0     0     0     0     0     0     0     X     X     0     X     0

     6|     0     0     0     0     0     0     0     0     0     0     X     X     0     X     0

     7|     0     0     0     0     0     0     0     0     0     0     X     X     0     X     0

     8|     0     0     0     0     0     0     0     0     0     0     X     X     0     X     0

     9|     0     0     0     0     0     0     0     0 